In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

In [79]:
data = pd.read_csv("data.csv")
target = "winner"

In [80]:
data.drop("B_ID", axis= 1, inplace= True)
data.drop("B_HomeTown", axis= 1, inplace= True)
data.drop("B_Name", axis= 1, inplace= True)
data.drop("B_Location", axis= 1, inplace= True)

data.drop("R_ID", axis= 1, inplace= True)
data.drop("R_HomeTown", axis= 1, inplace= True)
data.drop("R_Name", axis= 1, inplace= True)
data.drop("R_Location", axis= 1, inplace= True)

data.drop(["Event_ID"], axis=1, inplace=True)
data.drop(["Fight_ID"], axis=1, inplace=True)

In [81]:
for column in data:
    if("Round" in column):
        data[column].fillna(data[column].median())

data["B_Height"].fillna(data["B_Height"].mean())
data["B_Weight"].fillna(data["B_Weight"].mean())

data["R_Height"].fillna(data["R_Height"].mean())
data["R_Weight"].fillna(data["R_Weight"].mean())

data = data.fillna(method='bfill')

In [82]:
weightDf = data[["B_Weight", "R_Weight"]]
minWeight = weightDf.min().min()
maxWeight = weightDf.max().max()
bins = np.linspace(minWeight, maxWeight, num=5)
data["B_Weight"] = pd.cut(data["B_Weight"], bins=bins)
data["R_Weight"] = pd.cut(data["R_Weight"], bins=bins)

In [83]:
percantage = 0.3
categorical = data.select_dtypes(include="object").columns
for column in categorical:
    if column == target: continue
    values = data[column].value_counts().values
    qtile = np.quantile(values, percantage)
    grouped = data.groupby(column)[column].transform("size")
    data.loc[grouped <= qtile, column] = "other"
    dummies =  pd.get_dummies(data[column])
    data.drop(column, axis=1, inplace=True)
    data = pd.concat([data, dummies], axis=1)

In [71]:
train, test = train_test_split(data, test_size=0.2)
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)